In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
df=pd.read_csv("/home/sachchida/gitrepo/ml/data/spam-detection/spam.csv",encoding="ISO-8859-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [29]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [30]:
import re
def clean_text(text):
    # remove HTML
    text = re.sub(r'<.*?>', '', text)
    # replace URLs
    text = re.sub(r'http\S+|www\S+', ' URL ', text)
    # keep hashtag text
    text = re.sub(r'#(\w+)', r'\1', text)
    # lowercase
    text = text.lower()
    # remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [31]:
df['cleaned_text']=df['v2'].apply(clean_text)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned_text
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a wkly comp to win fa cup final ...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah i dont think he goes to usf he lives aroun...


In [32]:
from sklearn.preprocessing import LabelEncoder
label_enc=LabelEncoder()
df['encoded_label']=label_enc.fit_transform(df['v1'])
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,cleaned_text,encoded_label
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go until jurong point crazy available only in ...,0
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry in a wkly comp to win fa cup final ...,1
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,u dun say so early hor u c already then say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,nah i dont think he goes to usf he lives aroun...,0


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,2),
    stop_words='english',
)

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['cleaned_text'],df['encoded_label'],test_size=0.2,random_state=42)


In [35]:
X_train.head()

1978    no im in the same boat still here at my moms c...
3989    bank of granite issues strongbuy explosive pic...
3935        they r giving a second chance to rahul dengra
4078               o i played smash bros ltgt religiously
4086    private your account statement for shows unred...
Name: cleaned_text, dtype: object

In [36]:
X_train_vec=tfidf.fit_transform(X_train)
X_test_vec=tfidf.transform(X_test)


In [37]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs',max_iter=2000,class_weight='balanced')
model.fit(X_train_vec,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [38]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix
print(f"classification report",classification_report(y_test,y_pred))
print(f"Macro Average f1 score ",f1_score(y_test,y_pred,average="macro"))
print(f"Weighted f1 score ",f1_score(y_test,y_pred,average="weighted"))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred))

classification report               precision    recall  f1-score   support

           0       0.98      0.99      0.98       965
           1       0.91      0.89      0.90       150

    accuracy                           0.97      1115
   macro avg       0.94      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Macro Average f1 score  0.9419004543749219
Weighted f1 score  0.9730178087976422
Confusion Matrix  [[951  14]
 [ 16 134]]


In [39]:
logistic_regularized=LogisticRegression(
    solver="lbfgs",
    max_iter=2000,
    class_weight='balanced'
)

In [40]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 5, 10]  # Regularization strength
}

grid = GridSearchCV(
    logistic_regularized,
    param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train_vec,y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LogisticRegre...max_iter=2000)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.01, 0.1, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'f1_macro'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate param

In [41]:
grid.best_params_

{'C': 5}

In [42]:
best_model = grid.best_estimator_
y_pred=best_model.predict(X_test_vec)

In [43]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix
print(f"classification report",classification_report(y_test,y_pred))
print(f"Macro Average f1 score ",f1_score(y_test,y_pred,average="macro"))
print(f"Weighted f1 score ",f1_score(y_test,y_pred,average="weighted"))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred))

classification report               precision    recall  f1-score   support

           0       0.98      0.99      0.98       965
           1       0.91      0.89      0.90       150

    accuracy                           0.97      1115
   macro avg       0.94      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Macro Average f1 score  0.9419004543749219
Weighted f1 score  0.9730178087976422
Confusion Matrix  [[951  14]
 [ 16 134]]


### Adding more parameters for GridSearchCV


In [46]:
param_grid = {
    # Logistic Regression
    'C': [0.1, 1, 5, 10],
    'penalty': ['l2','l1'],
    'solver': ['lbfgs'],
    'class_weight': ['balanced']
}

In [47]:
grid = GridSearchCV(
    logistic_regularized,
    param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train_vec,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LogisticRegre...max_iter=2000)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [0.1, 1, ...], 'class_weight': ['balanced'], 'penalty': ['l2', 'l1'], 'solver': ['lbfgs']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'f1_macro'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is display

In [48]:
grid.best_params_

{'C': 5, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}

In [49]:
best_model=grid.best_estimator_

In [50]:
best_model = grid.best_estimator_
y_pred=best_model.predict(X_test_vec)

In [51]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix
print(f"classification report",classification_report(y_test,y_pred))
print(f"Macro Average f1 score ",f1_score(y_test,y_pred,average="macro"))
print(f"Weighted f1 score ",f1_score(y_test,y_pred,average="weighted"))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred))

classification report               precision    recall  f1-score   support

           0       0.98      0.99      0.98       965
           1       0.91      0.89      0.90       150

    accuracy                           0.97      1115
   macro avg       0.94      0.94      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Macro Average f1 score  0.9419004543749219
Weighted f1 score  0.9730178087976422
Confusion Matrix  [[951  14]
 [ 16 134]]


### Full Pipeline + GridSearchCV (TF-IDF + Logistic Regression)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        stop_words='english'
    )),
    ('clf', LogisticRegression(
        max_iter=2000,
        class_weight='balanced',
        solver='lbfgs'
    ))
])

# Parameter grid
param_grid = {
    # TF-IDF parameters
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__min_df': [1, 2, 5],
    'tfidf__max_df': [0.9, 0.95],
    'tfidf__sublinear_tf': [True],
    'tfidf__max_features': [None, 20000, 50000],

    # Logistic Regression regularization
    'clf__C': [0.1, 1, 5, 10]
}

# GridSearchCV
grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Fit
grid.fit(X_train, y_train)




In [ ]:
grid.best_params_

In [ ]:
# Best model
best_model = grid.best_estimator_

# Predict
y_pred = best_model.predict(X_test)